In [15]:
# DOCX 파일을 읽고 정규식 기준으로 청킹하여 JSONL로 저장하는 코드 (python-docx 사용)
import os
import re
import json
from docx import Document
from typing import List, Dict, Any

def custom_regex_split(text: str, pattern: str = r"^\<.+\>") -> List[str]:
    """
    정규식 패턴을 기준으로 텍스트를 청킹합니다.
    
    Args:
        text: 분할할 텍스트
        pattern: 청크 시작점을 나타내는 정규식 패턴
    
    Returns:
        청킹된 텍스트 리스트
    """
    # 줄별로 분할
    lines = text.split('\n')
    chunks = []
    current_chunk = []
    
    for line in lines:
        # 정규식 패턴에 매치되는 경우 새 청크 시작
        if re.match(pattern, line.strip()):
            # 이전 청크가 있으면 저장
            if current_chunk:
                chunks.append('\n'.join(current_chunk).strip())
            # 새 청크 시작
            current_chunk = [line]
        else:
            current_chunk.append(line)
    
    # 마지막 청크 추가
    if current_chunk:
        chunks.append('\n'.join(current_chunk).strip())
    
    return [chunk for chunk in chunks if chunk.strip()]  # 빈 청크 제거

def load_and_chunk_docx(docx_path: str) -> List[Dict[str, Any]]:
    """
    DOCX를 로드하고 정규식 기준으로 청킹합니다.
    
    Args:
        docx_path: DOCX 파일 경로
    
    Returns:
        청킹된 문서들의 리스트
    """
    # 파일 존재 확인
    if not os.path.isfile(docx_path):
        raise ValueError(f"파일이 존재하지 않습니다: {docx_path}")
        
    # DOCX 로드 (python-docx 사용)
    doc = Document(docx_path)
    
    # 모든 paragraph의 텍스트를 하나로 합침
    full_text = ""
    for paragraph in doc.paragraphs:
        full_text += paragraph.text + '\n'
    
    # 정규식 기준으로 청킹
    chunks = custom_regex_split(full_text)
    
    # chunks에서 첫줄을 제거 (줄넘김 기준으로 첫줄 제거)
    processed_chunks = []
    for chunk in chunks:
        lines = chunk.split('\n')
        if len(lines) > 1:
            # 첫 줄을 제거하고 나머지 줄들을 합침
            processed_chunk = '\n'.join(lines[1:])
            processed_chunks.append(processed_chunk)
        elif len(lines) == 1:
            # 한 줄짜리 청크는 빈 문자열로 처리
            processed_chunks.append('')
        else:
            processed_chunks.append('')
    
    chunks = processed_chunks
    
    # 결과 포맷팅
    result = []
    for i, chunk in enumerate(chunks):
        if chunk.strip():  # 빈 청크 제외
            result.append({
                "content": chunk,
                "length": len(chunk),
                "original_content": chunk
            })
    
    return result

def save_to_jsonl(data: List[Dict[str, Any]], output_path: str):
    """
    데이터를 JSONL 형태로 저장합니다.
    
    Args:
        data: 저장할 데이터 리스트
        output_path: 출력 파일 경로
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

# 메인 실행 코드
if __name__ == "__main__":
    # 현재 디렉토리 확인
    current_dir = os.getcwd()
    
    # 절대 경로로 DOCX 파일 경로 설정
    docx_path = os.path.abspath(os.path.join(current_dir, "..", "resource", "rag_data.docx"))
    output_path = "chunks_docx.jsonl"
    
    # 파일 존재 확인
    if not os.path.exists(docx_path):
        print(f"파일을 찾을 수 없습니다: {docx_path}")
        # 대안 경로들 시도
        alternative_paths = [
            os.path.join(current_dir, "resource", "rag_data.docx"),
            "/resource/국어 지식 기반 생성(RAG) 참조 문서.docx"
        ]
        
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                docx_path = alt_path
                print(f"파일을 찾았습니다: {docx_path}")
                break
        else:
            print("모든 경로에서 파일을 찾을 수 없습니다.")
            print(f"현재 디렉토리: {current_dir}")
            exit(1)
    
    print(f"DOCX 파일 경로: {docx_path}")
    print("DOCX 파일을 로드하고 청킹을 시작합니다...")
    
    # DOCX 로드 및 청킹
    chunks = load_and_chunk_docx(docx_path)
    
    print(f"총 {len(chunks)}개의 청크가 생성되었습니다.")
    
    # length 기준으로 내림차순 정렬
    chunks.sort(key=lambda x: x['length'], reverse=True)
    print("청크들을 length 기준으로 내림차순 정렬했습니다.")
    
    # JSONL로 저장
    save_to_jsonl(chunks, output_path)
    
    print(f"청크들이 {output_path}에 저장되었습니다.")
    
    # 첫 번째 청크 미리보기 (가장 긴 청크)
    if chunks:
        print(f"\n첫 번째 청크 미리보기 (가장 긴 청크):")
        print(f"길이: {chunks[0]['length']} 문자")
        print(f"내용 (처음 200자):")
        print(chunks[0]['content'][:200] + "..." if len(chunks[0]['content']) > 200 else chunks[0]['content'])


DOCX 파일 경로: /home/orin/Project/Korean_QA_RAG_2025/RAG/resource/rag_data.docx
DOCX 파일을 로드하고 청킹을 시작합니다...
총 115개의 청크가 생성되었습니다.
청크들을 length 기준으로 내림차순 정렬했습니다.
청크들이 chunks_docx.jsonl에 저장되었습니다.

첫 번째 청크 미리보기 (가장 긴 청크):
길이: 2101 문자
내용 (처음 200자):
한 가지 의미를 나타내는 형태 몇 가지가 널리 쓰이며 표준어 규정에 맞으면, 그 모두를 표준어로 삼는다.
- 복수 표준어: 가는-허리/잔-허리, 가락-엿/가래-엿, 가뭄/가물, 가엾다/가엽다, 감감-무소식/감감-소식, 개수-통/설거지-통, 개숫-물/설거지-물, 갱-엿/검은-엿, -거리다/-대다, 거위-배/횟-배, 것/해, 게을러-빠지다/게을러-터지다, 고깃...


In [16]:
import json
import os

def process_chunks(input_file, output_file):
    """
    chunks.jsonl의 각 라인을 읽어서 content를 재구성합니다.
    각 content의 첫 줄을 헤더로 하고, 나머지 줄들과 조합하여 새로운 청크들을 만듭니다.
    """
    new_chunks = []
    
    # 파일 경로 확인
    current_dir = os.path.dirname(os.path.abspath("__file__"))
    if not os.path.exists(input_file):
        print(f"파일을 찾을 수 없습니다: {input_file}")
        # 상대 경로 시도
        alternative_paths = [
            "chunks_docx.jsonl",
            os.path.join(current_dir, "chunks_docx.jsonl"),
            os.path.join(current_dir, "..", "00_rag_make_dataset", "chunks_docx.jsonl")
        ]
        
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                input_file = alt_path
                print(f"파일을 찾았습니다: {input_file}")
                break
        else:
            print("모든 경로에서 파일을 찾을 수 없습니다.")
            print(f"현재 디렉토리: {current_dir}")
            return
    
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():  # 빈 줄 무시
                    item = json.loads(line.strip())
                    content = item['content']
                    original_content = item['original_content']
                    
                    # 줄바꿈으로 분할
                    lines = content.split('\n')
                    
                    if len(lines) >= 2:  # 최소 2줄 이상이어야 처리
                        header = lines[0]  # 첫 줄을 헤더로
                        
                        # 나머지 각 줄과 헤더를 조합
                        for i, line_content in enumerate(lines[1:], 1):
                            if line_content.strip():  # 빈 줄 제외
                                new_content = header + '\n' + line_content
                                new_chunk = {
                                    'content': new_content,
                                    'length': len(new_content),
                                    'original_content': original_content
                                }
                                new_chunks.append(new_chunk)
        
        # 새로운 JSONL 파일로 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            for chunk in new_chunks:
                f.write(json.dumps(chunk, ensure_ascii=False) + '\n')
        
        print(f"원본 청크에서 {len(new_chunks)}개의 새로운 청크를 생성했습니다.")
        print(f"결과가 {output_file}에 저장되었습니다.")
        
        # 몇 개 예시 출력
        print("\n=== 처리 결과 예시 ===")
        for i, chunk in enumerate(new_chunks[:5]):  # 처음 5개만 출력
            print(f"\n청크 {i+1}:")
            print(f"길이: {chunk['length']}")
            print(f"내용:\n{chunk['content'][:200]}{'...' if len(chunk['content']) > 200 else ''}")
            print("-" * 50)
    except Exception as e:
        print(f"오류 발생: {e}")

# 주피터 노트북에서 실행할 때는 상대 경로 사용
input_file = "chunks_docx.jsonl"
output_file = "rechunked_data.jsonl"

process_chunks(input_file, output_file)

원본 청크에서 908개의 새로운 청크를 생성했습니다.
결과가 rechunked_data.jsonl에 저장되었습니다.

=== 처리 결과 예시 ===

청크 1:
길이: 2101
내용:
한 가지 의미를 나타내는 형태 몇 가지가 널리 쓰이며 표준어 규정에 맞으면, 그 모두를 표준어로 삼는다.
- 복수 표준어: 가는-허리/잔-허리, 가락-엿/가래-엿, 가뭄/가물, 가엾다/가엽다, 감감-무소식/감감-소식, 개수-통/설거지-통, 개숫-물/설거지-물, 갱-엿/검은-엿, -거리다/-대다, 거위-배/횟-배, 것/해, 게을러-빠지다/게을러-터지다, 고깃...
--------------------------------------------------

청크 2:
길이: 57
내용:
(1) 같은 자격의 어구를 열거할 때 그 사이에 쓴다.
- 근면, 검소, 협동은 우리 겨레의 미덕이다.
--------------------------------------------------

청크 3:
길이: 74
내용:
(1) 같은 자격의 어구를 열거할 때 그 사이에 쓴다.
- 충청도의 계룡산, 전라도의 내장산, 강원도의 설악산은 모두 국립 공원이다.
--------------------------------------------------

청크 4:
길이: 91
내용:
(1) 같은 자격의 어구를 열거할 때 그 사이에 쓴다.
- 집을 보러 가면 그 집이 내가 원하는 조건에 맞는지, 살기에 편한지, 망가진 곳은 없는지 확인해야 한다.
--------------------------------------------------

청크 5:
길이: 77
내용:
(1) 같은 자격의 어구를 열거할 때 그 사이에 쓴다.
다만, (가) 쉼표 없이도 열거되는 사항임이 쉽게 드러날 때는 쓰지 않을 수 있다.
--------------------------------------------------
